In [2]:
from cobra.io import load_model
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion,
    double_gene_deletion, double_reaction_deletion,
    flux_variability_analysis
)
from cobra.util.solver import linear_reaction_coefficients

model = load_model("iML1515")

In [ ]:
# Writing this displays information about the model
model

In [ ]:
model.optimize()

model.optimize() = Returns an object named Solution:
- objective_value 
- status 
- shadow_price = Susceptibilty of the solution to the metabolite in the question. Basically 'how much it would help the reaction if there were more of it?'

Shadow prices and status can be found as `solution.shadow_prices` and `solution.status`

In [ ]:
model.summary()
# Full summary of the model

In [ ]:
print(f"Normal medium {model.medium}")

with model:
    medium_No_Oxygen = model.medium
    medium_No_Oxygen["EX_o2_e"] = 0.0
    model.medium = medium_No_Oxygen
    print(f"No oxygen in medium: {model.medium}")

In [ ]:
o2_Intake = model.reactions.get_by_id("EX_o2_e")
print(f"Normal o2 intakes: {o2_Intake.bounds}")

with model:
    o2_Intake = model.reactions.get_by_id("EX_o2_e")
    o2_Intake.bounds = (0,1000)
    print(f"Changed bounds {o2_Intake.bounds}")

In [ ]:
model.metabolites.nadh_c.summary()
# This summary gives more information about each reaction that metabolite is used

In [ ]:
model.metabolites.get_by_id("nadh_c")
# More information about each reaction(copy ones) and location of the metabolite in cell
# e = extracellular
# c = cytosol

In [ ]:
with model:
    model.objective = "ATPS4rpp" # Objective is now producing ATP
    linerr = linear_reaction_coefficients(model)
    objective = model.optimize().objective_value
    print(f"{linerr} \n {objective}")

FVA = Flux variability analysis, calculates the maximum and minumum flux a reaction can have while satifying all condition and keeping the goal in mind.
- loopless : This prevents loops that are impossible to happen *in vivo*. Default is false. loopless=True takes more time than false
    Different numbers in loopless True/False means that reaction participates in a loop

In [ ]:
flux_variability_analysis(
    model,
    ["ATPS4rpp", "CTYK1", "PYK"],
    loopless=True
)

In [ ]:
flux_variability_analysis(
    model,
    model.reactions[:10],
    loopless=True
)

In [ ]:
flux_variability_analysis(
    model,
    model.reactions[:10],
    loopless=False
)

Genes and reactions can be knocked out with `.knock_out()` and `single_gene/reaction_deletion()`.
Pair of reactions or genes can be knocked out with `double_gene/reaction_deletion()`

`.knock_out()` cannot do multiple deletions in one go, but `single_gene/reaction_deletion()` can take a list as an argument 

In [ ]:
print(f"Regular solution: {model.optimize().objective_value}")

with model:
    model.reactions.ATPS4rpp.knock_out()
    print(f"Knock out of ATP synthase: {model.optimize().objective_value}")

with model:
    model.genes.b3739.knock_out() # Probably won't affect since there are too many possible GPR
    print(f"Knock out of ATP synthease gene: {model.optimize().objective_value}")

In [ ]:
print(f"Regular solution: {model.optimize().objective_value}")

with model:
    single_gene_deletion(model, ["b1779"])
    print(f"Knock out of GAPD gene: {model.optimize().objective_value}")

with model:
    single_reaction_deletion(model, ["GAPD"])
    print(f"Knock out of GAPD reaction: {model.optimize().objective_value}")